# Widgets in threads

Widget interactions are blocked when something is running in the kernel. I have not succeeded in modifying the kernel event loop to overcome this. So try threads. 

After https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Asynchronous.html#Updating-a-widget-in-the-background, with hacks:

# Setup

In [ ]:
import trio
import threading
from IPython.display import display
import ipywidgets as widgets
import math
import time

## A global state object

In [ ]:
class Thing:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
g = Thing()

## A function that does work in a thread
This will update a widget's value from a supplied function. It also runs a task at `g.threadwork`.

In [ ]:
def work(w, valfun=lambda t:0.5):

    async def track_value(w, valfun):
        while not g.stop_requested:
            w.value = valfun()
            await trio.sleep(0.1) #DEBUG

    async def worker():
        while not g.stop_requested:
            await trio.sleep(await g.threadwork())

    
    async def threadloop(w, valfun):
        async with trio.open_nursery() as nursery:
            nursery.start_soon(track_value, w, valfun)
            nursery.start_soon(worker)
        
    trio.run(threadloop, w, valfun)

## The widgets

In [ ]:
progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0)
s_w = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.01)
b_w = widgets.Button(
    description='stop',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='click to stop thread',
    icon='stop' # (FontAwesome names without the `fa-` prefix)
)
out_w = widgets.Output()

### and their behavior

In [ ]:
g.stop_requested = False
def request_stop(b):
    g.stop_requested = True
    #with out_w:
    #    print("stop requested")
b_w.on_click(request_stop)

display(progress, s_w, b_w, out_w)

# Create the background thread

In [ ]:
thread = threading.Thread(target=work, args=(progress, lambda: s_w.value))

## Set up a pretend job and start the thread

In [ ]:
g.work_ctr = 0
async def just_one():
    g.work_ctr += 1
    return 1
g.threadwork = just_one
g.stop_requested = False
thread.start()

## Testing
* Move the slider and see the progress bar track
* Examine g.work_ctr to see it growing
* Hit the stop button and see the tracking and work stops

## Join up if it's done, and examine work results

In [ ]:
thread.is_alive() or thread.join()

In [ ]:
print(f"{g.work_ctr}")

In [ ]:
async def fastly():
    g.work_ctr += 1
    return .01
g.threadwork = fastly

In [ ]:
g.work_ctr = 0

In [ ]:
import numpy as np
a = np.random.randn(2048,3072)

In [ ]:
%timeit a@a.T

## Burn cycles with `numpy`

In [ ]:
async def mmul():
    p = a @ a.T
    g.work_product = np.einsum('ij,ij', p, p)
    g.work_ctr += 1
    return 0
g.threadwork = mmul

In [ ]:
g.work_ctr

In [ ]:
async def worked_enough(v):
    if g.work_ctr >= v:
        g.threadwork = lambda: 1
    await trio.sleep(1)
    
trio.run(worked_enough, 100)

In [ ]:
g.threadwork

In [ ]:
threading.active_count()